## 获取stock数据范例

In [ ]:
import tushare as ts
token = '3eca2d2172110e537cdb74242105a1ba412bee6277a0133d3277a46b'
#ts.set_token(token)
#pro = ts.pro_api()
pro = ts.pro_api(token)

## 股票列表
>* 接口：stock_basic
>* 描述：获取基础信息数据，包括股票代码、名称、上市日期、退市日期等

**参数**
>- is_hs	str	是否沪深港通标的，N否 H沪股通 S深股通
>- list_status	str	上市状态： L上市 D退市 P暂停上市，默认L
>- exchange	str	交易所 SSE上交所 SZSE深交所 HKEX港交所(未上线)

**结果**
>- ts_code	str	TS代码
>- symbol	str	股票代码
>- name	str	股票名称
>- area	str	所在地域
>- industry	str	所属行业
>- fullname	str	股票全称
>- enname	str	英文全称
>- market	str	市场类型 （主板/中小板/创业板/科创板）
>- exchange	str	交易所代码
>- curr_type	str	交易货币
>- list_status	str	上市状态： L上市 D退市 P暂停上市
>- list_date	str	上市日期
>- delist_date	str	退市日期
>- is_hs	str	是否沪深港通标的，N否 H沪股通 S深股通

In [ ]:
# 查询当前所有正常上市交易的股票列表
data = pro.query('stock_basic', exchange='', list_status='L', 
                 fields='ts_code,symbol,name,area,industry,fullname,enname,market,exchange,curr_type,list_status,list_date,is_hs')
data.to_csv('data/stock_basic.csv', index=False)
data
#data.to_dict(orient='records')

## 交易日历
>* 接口：trade_cal
>* 描述：获取各大交易所交易日历数据,默认提取的是上交所

**输入参数**

>- 名称	类型	必选	描述
>- exchange	str	N	交易所 SSE上交所,SZSE深交所,CFFEX 中金所,SHFE 上期所,CZCE 郑商所,DCE 大商所,INE 上能源,IB 银行间,XHKG 港交所
>- start_date	str	N	开始日期
>- end_date	str	N	结束日期
>- is_open	str	N	是否交易 '0'休市 '1'交易

**输出参数**

>- 名称	类型	默认显示	描述
>- exchange	str	Y	交易所 SSE上交所 SZSE深交所
>- cal_date	str	Y	日历日期
>- is_open	str	Y	是否交易 0休市 1交易
>- pretrade_date	str	N	上一个交易日

In [ ]:
data = pro.query('trade_cal', exchange='', start_date='20210930', end_date='20211013')
#data.to_csv('data/trade_cal.csv', index=False)
data

## 上市公司基本信息
>* 接口：stock_company
>* 描述：获取上市公司基础信息，单次提取4000条，可以根据交易所分批提取

**输入参数**

>- 名称	类型	必须	描述
>- ts_code	str	N	股票代码
>- exchange	str	N	交易所代码 ，SSE上交所 SZSE深交所

**输出参数**

>- 名称	类型	默认显示	描述
>- ts_code	str	Y	股票代码
>- exchange	str	Y	交易所代码 ，SSE上交所 SZSE深交所
>- chairman	str	Y	法人代表
>- manager	str	Y	总经理
>- secretary	str	Y	董秘
>- reg_capital	float	Y	注册资本
>- setup_date	str	Y	注册日期
>- province	str	Y	所在省份
>- city	str	Y	所在城市
>- introduction	str	N	公司介绍
>- website	str	Y	公司主页
>- email	str	Y	电子邮件
>- office	str	N	办公室
>- employees	int	Y	员工人数
>- main_business	str	N	主要业务及产品
>- business_scope	str	N	经营范围

In [ ]:
qlist = [('SSE','data/stock_company_sse.csv'),('SZSE','data/stock_company_szse.csv')]
for q in qlist:
    data = pro.stock_company(exchange=q[0], 
            fields='ts_code,exchange,chairman,manager,secretary,reg_capital,setup_date,province,city,introduction,website,email,office,employees,main_business,business_scope')
    data.to_csv(q[1], index=False)

## IPO新股列表
>* 接口：new_share
>* 描述：获取新股上市列表数据
>* 限量：单次最大2000条，总量不限制

**输入参数**

>- 名称	类型	必选	描述
>- start_date	str	N	上网发行开始日期
>- end_date	str	N	上网发行结束日期

**输出参数**

>- 名称	类型	默认显示	描述
>- ts_code	str	Y	TS股票代码
>- sub_code	str	Y	申购代码
>- name	str	Y	名称
>- ipo_date	str	Y	上网发行日期
>- issue_date	str	Y	上市日期
>- amount	float	Y	发行总量（万股）
>- market_amount	float	Y	上网发行总量（万股）
>- price	float	Y	发行价格
>- pe	float	Y	市盈率
>- limit_amount	float	Y	个人申购上限（万股）
>- funds	float	Y	募集资金（亿元）
>- ballot	float	Y	中签率

In [ ]:
data = pro.new_share(start_date='20180101', end_date='20200731')
data.to_csv('data/new_share.csv', index=False)

## 日线行情
>* 接口：daily
>* 数据说明：交易日每天15点～16点之间。本接口是未复权行情，停牌期间不提供数据。
>* 调取说明：基础积分每分钟内最多调取500次，每次5000条数据，相当于23年历史，用户获得超过5000积分正常调取无频次限制。
>* 描述：获取股票行情数据，或通过通用行情接口获取数据，包含了前后复权数据。

**输入参数**

>- 名称	类型	必选	描述
>- ts_code	str	N	股票代码（支持多个股票同时提取，逗号分隔）
>- trade_date	str	N	交易日期（YYYYMMDD）
>- start_date	str	N	开始日期(YYYYMMDD)
>- end_date	str	N	结束日期(YYYYMMDD)
>- 注：日期都填YYYYMMDD格式，比如20181010

**输出参数**

>- 名称	类型	描述
>- ts_code	str	股票代码
>- trade_date	str	交易日期
>- open	float	开盘价
>- high	float	最高价
>- low	float	最低价
>- close	float	收盘价
>- pre_close	float	昨收价
>- change	float	涨跌额
>- pct_chg	float	涨跌幅 （未复权，如果是复权请用 通用行情接口 ）
>- vol	float	成交量 （手）
>- amount	float	成交额 （千元）

In [ ]:
# 取多个股票
slist = ['600358.SH','002123.SZ','600068.SH','002185.SZ']
for s in slist:
    data = pro.query('daily', ts_code=s, start_date='19900101', end_date='20200721')
    data.to_csv('data/daily-%s.csv' % s, index=False)

# 通过日期取历史某一天的全部历史
#data = pro.query('daily', trade_date='20200721')

In [ ]:
#获取股票1分钟数据
ts.pro_bar(ts_code='600000.SH',
                    freq='1min', 
                    start_date='2020-01-07 09:00:00', 
                    end_date='2020-01-08 17:00:00')